# **Загрузка данных**

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/Files/

/content/drive/My Drive/Colab Notebooks/Files


In [ ]:
lem_tfidf = pd.read_csv('hyperc_lem_tfidf.csv')

In [ ]:
valence = pd.read_csv('hyperc_lem_valence.csv')
valence['valence'] = valence['valence'].replace({'neut': 1, 'pos': 2, 'neg': 0})
valence_list = valence['valence'].tolist()

In [ ]:
melspectrogram_path = pd.read_csv('melspectrogram_path.csv', header=None)

In [ ]:
melspectrogram_path[0] = 'output_images' + melspectrogram_path[0]

In [ ]:
melspectrogram_path

,0
0,output_images/melspectrogram_0.png
1,output_images/melspectrogram_1.png
2,output_images/melspectrogram_2.png
3,output_images/melspectrogram_3.png
4,output_images/melspectrogram_4.png
...,...
829,output_images/melspectrogram_829.png
830,output_images/melspectrogram_830.png
831,output_images/melspectrogram_831.png
832,output_images/melspectrogram_832.png


In [ ]:
specto_images = []
for index, row in melspectrogram_path.iterrows():
    image = Image.open(row[0])
    specto_images.append(image)
    print('Сохранен файл №', index)

Сохранен файл № 0
Сохранен файл № 1
Сохранен файл № 2
Сохранен файл № 3
Сохранен файл № 4
Сохранен файл № 5
Сохранен файл № 6
Сохранен файл № 7
Сохранен файл № 8
Сохранен файл № 9
Сохранен файл № 10
Сохранен файл № 11
Сохранен файл № 12
Сохранен файл № 13
Сохранен файл № 14
Сохранен файл № 15
Сохранен файл № 16
Сохранен файл № 17
Сохранен файл № 18
Сохранен файл № 19
Сохранен файл № 20
Сохранен файл № 21
Сохранен файл № 22
Сохранен файл № 23
Сохранен файл № 24
Сохранен файл № 25
Сохранен файл № 26
Сохранен файл № 27
Сохранен файл № 28
Сохранен файл № 29
Сохранен файл № 30
Сохранен файл № 31
Сохранен файл № 32
Сохранен файл № 33
Сохранен файл № 34
Сохранен файл № 35
Сохранен файл № 36
Сохранен файл № 37
Сохранен файл № 38
Сохранен файл № 39
Сохранен файл № 40
Сохранен файл № 41
Сохранен файл № 42
Сохранен файл № 43
Сохранен файл № 44
Сохранен файл № 45
Сохранен файл № 46
Сохранен файл № 47
Сохранен файл № 48
Сохранен файл № 49
Сохранен файл № 50
Сохранен файл № 51
Сохранен файл № 52
Сох

# **Текст**

In [ ]:
#импорт библиотек
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
import scipy
import torch.utils.data as data_utils
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import torch.optim as optim
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Bidirectional, GRU
from keras.optimizers import Adam

# Задайте размеры признаков tfidf и целевой переменной
input_dim = 1000
output_dim = 1 # Размер целевой переменной valence

# Создание модели
model = Sequential()

# Добавление слоя BiGRU
model.add(Bidirectional(GRU(units=64, return_sequences=True), input_shape=(None, input_dim)))
model.add(Dropout(0.5))

# Добавление второго слоя BiGRU
model.add(Bidirectional(GRU(units=32)))
model.add(Dropout(0.5))

# Выходной слой
model.add(Dense(output_dim, activation='linear'))

# Компиляция модели
optimizer = Adam(lr=0.001)
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

# Вывод структуры модели
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_3 (Bidirecti  (None, None, 128)         409344    
 onal)                                                           
                                                                 
 dropout (Dropout)           (None, None, 128)         0         
                                                                 
 bidirectional_4 (Bidirecti  (None, 64)                31104     
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 440513 (1.68 MB)
Trainable params: 44051

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'], run_eagerly=True)

In [ ]:
import tensorflow as tf

# Очистка текущей сессии TensorFlow
tf.keras.backend.clear_session()

# Обучение модели
history = model.fit(X_text_train_tensor, y_train_tensor, epochs=10, batch_size=64, validation_split=0.3)

# Оценка модели на тестовых данных
loss, mae = model.evaluate(X_text_test_tensor, y_test_tensor)

print(loss, mae)

Epoch 1/10
7/7 [==============================] - 3s 395ms/step - loss: 1.0629 - mae: 0.9846 - val_loss: 0.8475 - val_mae: 0.8879
Epoch 2/10
7/7 [==============================] - 3s 340ms/step - loss: 0.7767 - mae: 0.8340 - val_loss: 0.5385 - val_mae: 0.6960
Epoch 3/10
7/7 [==============================] - 2s 373ms/step - loss: 0.4457 - mae: 0.6038 - val_loss: 0.2258 - val_mae: 0.4077
Epoch 4/10
7/7 [==============================] - 2s 295ms/step - loss: 0.1928 - mae: 0.3238 - val_loss: 0.0900 - val_mae: 0.1827
Epoch 5/10
7/7 [==============================] - 3s 386ms/step - loss: 0.1598 - mae: 0.2922 - val_loss: 0.1107 - val_mae: 0.2411
Epoch 6/10
7/7 [==============================] - 2s 235ms/step - loss: 0.1654 - mae: 0.2999 - val_loss: 0.0833 - val_mae: 0.1669
Epoch 7/10
7/7 [==============================] - 2s 242ms/step - loss: 0.1068 - mae: 0.2213 - val_loss: 0.0872 - val_mae: 0.1563
Epoch 8/10
7/7 [==============================] - 1s 171ms/step - loss: 0.1180 - mae: 0.22

In [ ]:
from sklearn.metrics import accuracy_score

# Получение предсказаний модели
y_pred = model.predict(X_text_test_tensor)

# Преобразование предсказаний в метки классов (например, с помощью порогового значения)
# Например, если вы решаете задачу бинарной классификации, вы можете использовать порог 0.5
y_pred_classes = (y_pred > 0.5).astype(int)

# Вычисление accuracy
accuracy = accuracy_score(y_test_tensor, y_pred_classes)
print("Accuracy:", accuracy)


8/8 [==============================] - 1s 90ms/step
Accuracy: 0.8844621513944223


In [ ]:
from sklearn.metrics import f1_score

# Вычисление F1-меры
f1 = f1_score(y_test_tensor, y_pred_classes, average='weighted')
print("F1-score:", f1)


F1-score: 0.8302350850298593


In [ ]:
import tensorflow as tf

# Преобразование массивов NumPy в тензоры
X_text_train_tensor = tf.convert_to_tensor(X_text_train, dtype=tf.float32)
X_text_test_tensor = tf.convert_to_tensor(X_text_test, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.float32)

# Предполагая, что X_text_train_tensor и X_text_test_tensor имеют форму (num_samples, num_features)

# Определите количество временных шагов
timesteps = X_text_train_tensor.shape[0]  # Изменение в этой строке

# Измените форму входных тензоров, чтобы добавить размерность временного шага и батчей
X_text_train_tensor = tf.reshape(X_text_train_tensor, (-1, 1, input_dim))  # Изменение в этой строке
X_text_test_tensor = tf.reshape(X_text_test_tensor, (-1, 1, input_dim))  # Изменение в этой строке


In [ ]:
X_text_train, X_text_test, y_train, y_test = train_test_split(lem_tfidf.to_numpy(), valence_list, test_size=0.3, random_state=28)

In [ ]:
# Преобразование в тензоры
inputs_train = torch.tensor(X_text_train).float()
targets_train = torch.LongTensor(y_train)

inputs_test = torch.tensor(X_text_test).float()
targets_test = torch.LongTensor(y_test)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True)
testset = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(1000, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.tanh(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(1000, 1000)
        self.bn1 = nn.BatchNorm1d(1000)
        self.dropout1 = nn.Dropout(0.8)

        self.fc2 = nn.Linear(1000, 600)
        self.bn2 = nn.BatchNorm1d(600)
        self.dropout2 = nn.Dropout(0.8)

        self.fc3 = nn.Linear(600, 600)
        self.bn3 = nn.BatchNorm1d(600)
        self.dropout3 = nn.Dropout(0.5)

        self.fc4 = nn.Linear(600, 1200)
        self.bn4 = nn.BatchNorm1d(1200)
        self.dropout4 = nn.Dropout(0.5)

        self.out = nn.Linear(1200, 3)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)

        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)

        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout3(x)

        x = F.relu(self.bn4(self.fc4(x)))
        x = self.dropout4(x)

        x = self.out(x)
        return F.log_softmax(x, dim=1)

In [ ]:
model_text = MLP()

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model_text.parameters(), lr=0.001)

In [ ]:
# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    model_text.train()
    running_loss = 0.0
    for inputs, labels in trainset:
        optimizer.zero_grad()
        outputs = model_text(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(trainset)}")

Epoch 1/10, Loss: 1.0785116490564848
Epoch 2/10, Loss: 0.6107727336256128
Epoch 3/10, Loss: 0.35304265234031174
Epoch 4/10, Loss: 0.33521156954137904
Epoch 5/10, Loss: 0.3417491120727439
Epoch 6/10, Loss: 0.29568916480792196
Epoch 7/10, Loss: 0.2563340107077046
Epoch 8/10, Loss: 0.18988652331264397
Epoch 9/10, Loss: 0.1540649429355797
Epoch 10/10, Loss: 0.13638352700754217


In [ ]:
model_text.eval()  # Устанавливаем модель в режим оценки (выключаем режим обучения)

all_predictions = []
all_targets = []

with torch.no_grad():
    for inputs, labels in testset:
        outputs = model_text(inputs.float())
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

acc_text = accuracy_score(all_targets, all_predictions)
f1_text = f1_score(all_targets, all_predictions, average='weighted')
#print(classification_report(all_targets, all_predictions))
print(acc_text, f1_text)

0.8725099601593626 0.8477046064109577


# **Аудио**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
specto_images = np.array(specto_images)
valence = np.array(valence)

In [ ]:
def resize_and_drop_channel(images, new_height, new_width):
    resized_images = []
    for img in images:
        pil_img = Image.fromarray(img)
        # Отбрасываем четвертый канал, если он есть
        if pil_img.mode == 'RGBA':
            pil_img = pil_img.convert('RGB')
        resized_img = pil_img.resize((new_width, new_height))
        resized_images.append(np.array(resized_img))
    return np.array(resized_images)

In [ ]:
new_height = 256
new_width = 256
resized_specto_images = resize_and_drop_channel(specto_images, new_height, new_width)

In [ ]:
num_classes = len(np.unique(valence))

In [ ]:
X_audio_train, X_audio_test, y_train, y_test = train_test_split(resized_specto_images, valence, test_size=0.3, random_state=28)

In [ ]:
# Создание архитектуры CNN
model_audio = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

In [ ]:
model_audio.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model_audio.fit(X_audio_train, y_train, epochs=4, batch_size=32, validation_data=(X_audio_test, y_test))

Epoch 1/4
19/19 [==============================] - 80s 4s/step - loss: 557.7016 - accuracy: 0.7650 - val_loss: 0.6319 - val_accuracy: 0.8845
Epoch 2/4
19/19 [==============================] - 78s 4s/step - loss: 0.4031 - accuracy: 0.9005 - val_loss: 0.7495 - val_accuracy: 0.8765
Epoch 3/4
19/19 [==============================] - 77s 4s/step - loss: 0.2970 - accuracy: 0.9417 - val_loss: 1.0087 - val_accuracy: 0.8765
Epoch 4/4
19/19 [==============================] - 81s 4s/step - loss: 0.1513 - accuracy: 0.9537 - val_loss: 1.0289 - val_accuracy: 0.8845


In [ ]:
from sklearn.metrics import f1_score

y_audio_pred = model_audio.predict(X_audio_test)
y_audio_pred_classes = np.argmax(y_audio_pred, axis=1)

f1_score(y_test, y_audio_pred_classes, average='weighted')

8/8 [==============================] - 9s 1s/step


0.8302350850298593

# **Объединение**

In [ ]:
lem_tfidf_tensor = torch.tensor(lem_tfidf.to_numpy(), dtype=torch.float32)

model_text.eval()

with torch.no_grad():
    Text_data = model_text(lem_tfidf_tensor)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
standardized_text_data = scaler.fit_transform(Text_data.numpy())

In [ ]:
df_text_data = pd.DataFrame(standardized_text_data)
df_text_data

,0,1,2
0,-0.000678,-0.219193,0.244902
1,-0.016629,-1.598349,0.898852
2,-0.147287,-0.402763,0.378504
3,-0.893441,-0.169164,0.416652
4,-0.331874,0.649806,-0.204636
...,...,...,...
829,0.052891,1.874017,-1.367889
830,-0.177929,0.468127,-0.123746
831,0.102858,0.838003,-0.462232
832,-0.565455,0.654226,-0.147668


In [ ]:
Audio_data = model_audio.predict(resized_specto_images)

27/27 [==============================] - 28s 1s/step


In [ ]:
standardized_audio_data = scaler.fit_transform(Audio_data)

In [ ]:
df_audio_data = pd.DataFrame(standardized_audio_data)
df_audio_data

,0,1,2
0,-0.449510,0.530261,-0.425737
1,-0.376127,-3.752329,4.868342
2,1.329451,-1.204552,0.806258
3,0.058178,-0.215450,0.237911
4,-0.065741,0.232913,-0.255710
...,...,...,...
829,-0.444499,0.527611,-0.425052
830,-0.432839,0.493412,-0.388553
831,-0.412379,0.387406,-0.267235
832,1.053704,-1.108447,0.830450


In [ ]:
df_outputs = pd.concat([df_text_data, df_audio_data], axis=1)
df_outputs

,0,1,2,0,1,2
0,-0.000678,-0.219193,0.244902,-0.449510,0.530261,-0.425737
1,-0.016629,-1.598349,0.898852,-0.376127,-3.752329,4.868342
2,-0.147287,-0.402763,0.378504,1.329451,-1.204552,0.806258
3,-0.893441,-0.169164,0.416652,0.058178,-0.215450,0.237911
4,-0.331874,0.649806,-0.204636,-0.065741,0.232913,-0.255710
...,...,...,...,...,...,...
829,0.052891,1.874017,-1.367889,-0.444499,0.527611,-0.425052
830,-0.177929,0.468127,-0.123746,-0.432839,0.493412,-0.388553
831,0.102858,0.838003,-0.462232,-0.412379,0.387406,-0.267235
832,-0.565455,0.654226,-0.147668,1.053704,-1.108447,0.830450


In [ ]:
X_ouputs_train, X_outputs_test, y_train, y_test = train_test_split(df_outputs.to_numpy(), valence_list, test_size=0.3, random_state=28)

In [ ]:
SGD_clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=1000)
SGD_clf.fit(X_ouputs_train, y_train)

y_pred_SGD = SGD_clf.predict(X_outputs_test)

print(classification_report(y_test, y_pred_SGD))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.88      0.96      0.92       222
           2       0.00      0.00      0.00        21

    accuracy                           0.85       251
   macro avg       0.29      0.32      0.31       251
weighted avg       0.78      0.85      0.82       251



In [ ]:
f1_score(y_test, y_pred_SGD, average='weighted')

0.8158400879241654

# **Результаты**

In [ ]:
#преобразование в тензоры
inputs_train = torch.tensor(X_outputs_train)
targets_train = torch.IntTensor(y_train)

inputs_test = torch.tensor(X_outputs_test)
targets_test = torch.IntTensor(y_test)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True)
testset = torch.utils.data.DataLoader(test, shuffle=False)

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(6, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 3)

    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.tanh(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

In [ ]:
model = MLP()

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainset:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(trainset)}")

Epoch 1/10, Loss: 0.0963952500280472
Epoch 2/10, Loss: 0.02832524131046605
Epoch 3/10, Loss: 0.012026809281441918
Epoch 4/10, Loss: 0.013720041158885454
Epoch 5/10, Loss: 0.01145171055772524
Epoch 6/10, Loss: 0.010066809225473018
Epoch 7/10, Loss: 0.008896144602847513
Epoch 8/10, Loss: 0.008478684541595602
Epoch 9/10, Loss: 0.013790108517564532
Epoch 10/10, Loss: 0.010771528357593183


In [ ]:
model.eval()  # Устанавливаем модель в режим оценки (выключаем режим обучения)

all_predictions = []
all_targets = []

with torch.no_grad():
    for inputs, labels in testset:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

acc_outputs = accuracy_score(all_targets, all_predictions)
f1_outputs = f1_score(all_targets, all_predictions, average='weighted')
#print(classification_report(all_targets, all_predictions))
print(acc_outputs, f1_outputs)

0.8804780876494024 0.8429642276059185


# **Эксперименты для повышения результата**

In [ ]:
from sklearn.model_selection import train_test_split
X_text_train, X_text_test, y_train, y_test = train_test_split(lem_tfidf.to_numpy(), valence_list, test_size=0.3, random_state=28)

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [ ]:
SGD_clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=1000)
SGD_clf.fit(X_text_train, y_train)

y_pred_SGD = SGD_clf.predict(X_text_test)

print(classification_report(y_test, y_pred_SGD))

              precision    recall  f1-score   support

           0       1.00      0.25      0.40         8
           1       0.90      0.98      0.94       222
           2       0.43      0.14      0.21        21

    accuracy                           0.89       251
   macro avg       0.78      0.46      0.52       251
weighted avg       0.86      0.89      0.86       251



In [ ]:
f1_score(y_test, y_pred_SGD, average='weighted')

0.8617667261986536

In [ ]:
Audio_data = model_audio.predict(resized_specto_images)

27/27 [==============================] - 40s 1s/step


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
standardized_audio_data = scaler.fit_transform(Audio_data)

In [ ]:
df_audio_data = pd.DataFrame(standardized_audio_data)
df_audio_data

,0,1,2
0,-0.449510,0.530261,-0.425737
1,-0.376127,-3.752329,4.868342
2,1.329451,-1.204552,0.806258
3,0.058178,-0.215450,0.237911
4,-0.065741,0.232913,-0.255710
...,...,...,...
829,-0.444499,0.527611,-0.425052
830,-0.432839,0.493412,-0.388553
831,-0.412379,0.387406,-0.267235
832,1.053704,-1.108447,0.830450


In [ ]:
Text_data = SGD_clf.predict(lem_tfidf)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SGDClassifier was fitted without feature names
  warnings.warn(


In [ ]:
df_text_data = pd.DataFrame(Text_data)
df_text_data

,0
0,1
1,2
2,1
3,1
4,1
...,...
829,1
830,1
831,1
832,1


In [ ]:
standardized_text_data = scaler.fit_transform(df_text_data)

In [ ]:
standardized_text_data = pd.DataFrame(standardized_text_data)

In [ ]:
df_final = pd.concat([standardized_text_data, df_audio_data], axis=1)
df_final

,0,0,1,2
0,-0.11976,-0.449510,0.530261,-0.425737
1,3.87543,-0.376127,-3.752329,4.868342
2,-0.11976,1.329451,-1.204552,0.806258
3,-0.11976,0.058178,-0.215450,0.237911
4,-0.11976,-0.065741,0.232913,-0.255710
...,...,...,...,...
829,-0.11976,-0.444499,0.527611,-0.425052
830,-0.11976,-0.432839,0.493412,-0.388553
831,-0.11976,-0.412379,0.387406,-0.267235
832,-0.11976,1.053704,-1.108447,0.830450


In [ ]:
X_ustal_train, X_ustal_test, y_train, y_test = train_test_split(df_final.to_numpy(), valence_list, test_size=0.3, random_state=28)

In [ ]:
from sklearn.svm import SVC

In [ ]:
svm = SVC(kernel='rbf')
svm.fit(X_ustal_train, y_train)

y_pred_svm = svm.predict(X_ustal_test)

print(classification_report(y_test, y_pred_svm))

              precision    recall  f1-score   support

           0       0.67      0.25      0.36         8
           1       0.90      0.98      0.94       222
           2       0.43      0.14      0.21        21

    accuracy                           0.89       251
   macro avg       0.67      0.46      0.51       251
weighted avg       0.86      0.89      0.86       251



In [ ]:
f1_score(y_test, y_pred_svm, average='weighted')

0.8624027354160816

In [ ]:
#преобразование в тензоры
inputs_train = torch.tensor(X_ustal_train)
targets_train = torch.IntTensor(y_train)

inputs_test = torch.tensor(X_ustal_test)
targets_test = torch.IntTensor(y_test)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True)
testset = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(4, 1000)
        self.bn1 = nn.BatchNorm1d(1000)
        self.dropout1 = nn.Dropout(0.8)

        self.fc2 = nn.Linear(1000, 600)
        self.bn2 = nn.BatchNorm1d(600)
        self.dropout2 = nn.Dropout(0.8)

        self.fc3 = nn.Linear(600, 600)
        self.bn3 = nn.BatchNorm1d(600)
        self.dropout3 = nn.Dropout(0.5)

        self.fc4 = nn.Linear(600, 1200)
        self.bn4 = nn.BatchNorm1d(1200)
        self.dropout4 = nn.Dropout(0.5)

        self.out = nn.Linear(1200, 3)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)

        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)

        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout3(x)

        x = F.relu(self.bn4(self.fc4(x)))
        x = self.dropout4(x)

        x = self.out(x)
        return F.log_softmax(x, dim=1)

In [ ]:
model = MLP()

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001)

In [ ]:
# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainset:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(trainset)}")

Epoch 1/10, Loss: 0.9236161865686116
Epoch 2/10, Loss: 0.47738017377100495
Epoch 3/10, Loss: 0.3237239475312986
Epoch 4/10, Loss: 0.2616937285975406
Epoch 5/10, Loss: 0.27680932534368413
Epoch 6/10, Loss: 0.23878393949646698
Epoch 7/10, Loss: 0.18704374997239365
Epoch 8/10, Loss: 0.17691288064969213
Epoch 9/10, Loss: 0.20027059042140058
Epoch 10/10, Loss: 0.14693955724176608


In [ ]:
model.eval()  # Устанавливаем модель в режим оценки (выключаем режим обучения)

all_predictions = []
all_targets = []

with torch.no_grad():
    for inputs, labels in testset:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

acc_outputs = accuracy_score(all_targets, all_predictions)
f1_outputs = f1_score(all_targets, all_predictions, average='weighted')
#print(classification_report(all_targets, all_predictions))
print(acc_outputs, f1_outputs)

0.8844621513944223 0.8302350850298593
